In [1]:
import chromadb
import os
import subprocess
from dotenv import load_dotenv
load_dotenv()

os.environ['LANGCHAIN_TRACING_V2'] = "true"
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ['TAVILY_API_KEY'] = os.getenv("TAVILY_API_KEY")
os.environ['GOOGLE_API_KEY'] = os.getenv("GEMINI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

import pandas as pd
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_groq import ChatGroq
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import StructuredQueryOutputParser, get_query_constructor_prompt
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

embed_model = OllamaEmbeddings(model=os.getenv('OLLAMA_EMBED'))
llama = ChatGroq(model=os.getenv("LLAMA_70B_GROQ"))
gemini = ChatGoogleGenerativeAI(model=os.getenv("GEMINI_FLASH"))

c:\AGENTS\agents_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### REVIEW COMPLAINTS STATUS

In [2]:
db = SQLDatabase.from_uri("sqlite:///skincare_complaints.db")


In [4]:
db.run("SELECT * FROM customer_complaints LIMIT 5")

"[(1, 'Alice Johnson', 'alice.johnson@example.com', 'Anti-Aging Cream', 'Caused skin irritation.', '2024-11-01', 'Open'), (2, 'Bob Smith', 'bob.smith@example.com', 'Sunscreen SPF 50', 'Leaves a white residue.', '2024-11-02', 'In Progress'), (3, 'Charlie Brown', 'charlie.brown@example.com', 'Moisturizing Lotion', 'Bottle was half empty.', '2024-11-03', 'Resolved'), (4, 'Daisy Parker', 'daisy.parker@example.com', 'Acne Treatment Gel', 'No visible results after 3 weeks.', '2024-11-04', 'Open'), (5, 'Evan Taylor', 'evan.taylor@example.com', 'Facial Cleanser', 'Strong chemical smell.', '2024-11-05', 'In Progress')]"

In [18]:
agent_executer = create_sql_agent(llama, db=db, 
                                  agent_type='zero-shot-react-description', 
                                  verbose=True
                                  )

In [23]:
res = agent_executer.invoke(
    "check if there is an entry containing email as 'alice.johnson@example.com' and status as open."
    "If entry exists, then return a summary of the entire record containing customer name," 
    "the product, complaint details and status"
    "If there exists no entry then only return Not Found")



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: customer_complaintsNow that I have the list of tables, I can query the schema of the `customer_complaints` table to see what columns it has.

Action: sql_db_schema
Action Input: customer_complaints
CREATE TABLE customer_complaints (
	complaint_id INTEGER, 
	customer_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	product_name TEXT NOT NULL, 
	complaint_details TEXT NOT NULL, 
	complaint_date TEXT NOT NULL, 
	status TEXT DEFAULT 'Open', 
	PRIMARY KEY (complaint_id)
)

/*
3 rows from customer_complaints table:
complaint_id	customer_name	email	product_name	complaint_details	complaint_date	status
1	Alice Johnson	alice.johnson@example.com	Anti-Aging Cream	Caused skin irritation.	2024-11-01	Open
2	Bob Smith	bob.smith@example.com	Sunscreen SPF 50	Leaves a white residue.	2024

In [24]:
res

{'input': "check if there is an entry containing email as 'alice.johnson@example.com' and status as open.If entry exists, then return a summary of the entire record containing customer name,the product, complaint details and statusIf there exists no entry then only return Not Found",
 'output': 'Alice Johnson, Anti-Aging Cream, Caused skin irritation., Open'}

### COMPLAINTS REGISTRATION --> CHECK IF ORDER EXISTS AND THEN ONLY REGISTER THE COMPLAINT

### 1. Entity Extractor from Email

In [ ]:
order_db = SQLDatabase.from_uri("sqlite:///skincare_orders.db")


In [29]:
entity_extractor_prompt = """
<|begin_of_text|><|start_header_id|>user<|end_header_id|>You are a helpful AI Agent, proficient in extracting required entities from an email that is related to a online order complaint.
Below are the mandatory entities needed in an input email body:
- order_id: the Order ID of the online purchase
- customer_email: customer's email id 
- complaint_details: A precise summary of the customer's complaint not more that 50 words


{format_instructions}

Now, Return the JSON output for the below email:

{email}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>
"""




In [30]:
from pydantic import BaseModel, Field

class email_entity(BaseModel):
    order_id: int = Field(...,description="The order ID of the product")
    customer_emai: str = Field(...,description="The email ID of the customer")
    product_name: str = Field(...,description="Name of the product that the user is complaining for")
    complaint_details: str = Field(...,description="A summary of the customer complaint in the not more that 30 words.")

In [31]:
parser = JsonOutputParser(pydantic_object=email_entity)

In [32]:


prompt_template = PromptTemplate(template=entity_extractor_prompt, input_variables=["email"], 
                                 partial_variables={"format_instructions":parser.get_format_instructions()},)


In [33]:
chain = prompt_template | llama | parser

In [27]:
sample_email = """
I am writing about a complaint regarding a skincare product I purchased from sephora website on 24th November.
The product cause rashes and skinn irritation in my face and I had to visit the doctor for the same.
I want to return and have a refund for the product. 
Below are the product details:
order_id: 3
email: eve@example.com
product_name: ABC lotion
"""

chain.invoke({"email": sample_email})

{'order_id': 3,
 'customer_email': 'eve@example.com',
 'product_name': 'ABC lotion',
 'complaint_details': 'The product cause rashes and skinn irritation in my face and I had to visit the doctor for the same.'}

In [ ]:
# Returns None when something is not present
sample_email = """
I am writing about a complaint regarding a skincare product I purchased from sephora website on 24th November.
The product cause rashes and skinn irritation in my face and I had to visit the doctor for the same.
I want to return and have a refund for the product. 
Below are the product details:
email: eve@example.com
product_name: ABC lotion
"""

chain.invoke({"email": sample_email})

{'order_id': None,
 'customer_email': 'eve@example.com',
 'product_name': 'ABC lotion',
 'complaint_details': 'Product causes rashes and skin irritation on face, had to visit doctor.'}

### 2. Router

In [1]:
from CustomerCaseAgent import build_graph
from pydantic_classes import AgentState

In [2]:
agent = build_graph()

sample_email = """
    I am writing about a complaint regarding a skincare product I purchased from sephora website on 24th November.
    The product cause rashes and skinn irritation in my face and I had to visit the doctor for the same.
    I want to return and have a refund for the product. 
    Below are the product details:
    order_id: 1
    email: eve@example.com
    product_name: ABC lotion
    """
state = AgentState(input_email = sample_email)

response = agent.invoke(state)

> Email Entity Extraction
{'order_id': 1, 'customer_email': 'eve@example.com', 'product_name': 'ABC lotion'} 


> Email Classification
input_email='\n    I am writing about a complaint regarding a skincare product I purchased from sephora website on 24th November.\n    The product cause rashes and skinn irritation in my face and I had to visit the doctor for the same.\n    I want to return and have a refund for the product. \n    Below are the product details:\n    order_id: 1\n    email: eve@example.com\n    product_name: ABC lotion\n    ' intent='New Complaint' customer_email='eve@example.com' order_id=1 complaint_summary=None product_name='ABC lotion' action=None feedback=None response=None products=None 


> New Complaint
1


OperationalError: no such column: order_id

In [3]:
# from IPython.display import Image, display
# from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

# display(
#     Image(
#         agent.get_graph().draw_mermaid_png(
#             draw_method=MermaidDrawMethod.API,
#         )
#     )
# )

In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect(database="online_store.db")
cur = conn.cursor()

In [7]:
res = cur.execute("SELECT * FROM orders WHERE order_id<6")

In [8]:
def fetch_one_as_dict(cursor):
    """
    Converts cursor.fetchone() result into a dictionary.
    """
    row = cursor.fetchone()
    if row is None:
        return None
    # Extract column names from cursor.description
    column_names = [desc[0] for desc in cursor.description]
    # Return a dictionary mapping column names to values
    return dict(zip(column_names, row))

In [9]:
fetch_one_as_dict(res)

{'order_id': 1,
 'customer_email': 'alice@example.com',
 'order_date': '2024-11-20',
 'order_amount': 50.25,
 'item_quantity': 2,
 'status': 'D'}